In [14]:
import os
import math
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances

k = 5
bins = 256
files_count = 1240
folder_path = 'F:\PR_Term_Project\CASIA-B'
hog_hist = np.zeros((files_count, bins), dtype=np.float)
mbhx_hist = np.zeros((files_count, bins), dtype=np.float)
mbhy_hist = np.zeros((files_count, bins), dtype=np.float)
# below we will create an array to store file label, file type and file subtype with convention such that
# labels will be same as filename before nm, bg, cl characters and file types will be represented as
# 1 -> nm, 2 -> bg and 3 -> cl and file subtype will be same as numbers after characters nm, bg, and cl
labels_array = np.zeros((files_count, 3), dtype=np.int)

def perform_encoding(folder_path, sour_file, feature_arr, dimen_tuple, flag=False, labels_array=-1):
    # flag and labels_array arguments are used for computing labels only once
    file_handle = open(sour_file, 'r')
    centroids = np.genfromtxt(file_handle, dtype=np.float)
    file_handle.close()
    
    # creating numpy array with labels and concatenating with centroids columnwise
    labels = np.array(range(bins)).reshape(-1, 1)
    centroids_labels = np.hstack((labels, centroids))

    j = 0
    for filename in os.listdir(folder_path):
        com_file_path = os.path.join(folder_path,  filename)
        file_handle = open(com_file_path, 'r')
        vid_arr = np.genfromtxt(file_handle, dtype=np.float)
        file_handle.close()
        
        # a check below to extract particular type of columns/feature from row/interest point
        if dimen_tuple[1] != -1:
            points = vid_arr[:, dimen_tuple[0]:dimen_tuple[1]]
        else:
            points = vid_arr[:, dimen_tuple[0]:]

        for i in range(points.shape[0]):
            single_point = points[i, :].reshape((-1, 96))

            distances = euclidean_distances(centroids, single_point)
            # concatenating the distances with centroids_labels columnwise because we need centroids and labels later
            closest_centroids = np.hstack((distances, centroids_labels))
            # sorting the array with respect to distances
            closest_centroids = closest_centroids[closest_centroids[:, 0].argsort()]
            closest_centroids = closest_centroids[:k, :]
            # get the bin no which will be same as label
            bin_no = closest_centroids[0, 1]
            bin_no = math.floor(bin_no)
            # applying salient coding formula to compute vote weight
            vote = closest_centroids[0, 0] / ((1 / (k - 1)) * np.sum(closest_centroids[1:, 0]))
            # following array will store votes and this array is being passed as argument to method
            feature_arr[j, bin_no] = feature_arr[j, bin_no] + vote
            # a check to get file labels, file types and file subtypes and is being executed only once
            if flag == True:
                # following array will store labels and this array is being passed as argument to method
                parts = filename.split('-')
                labels_array[j, 0] = int(parts[0])
                labels_array[j, 2] = int(parts[2])
                if parts[1] == 'nm':
                    labels_array[j, 1] = 1
                elif parts[1] == 'bg':
                    labels_array[j, 1] = 2
                else:
                    labels_array[j, 1] = 3

        print("Processed file# ", j)
        j = j + 1


In [15]:
centroids_hog_handle = 'F:\PR_Term_Project\Computed_features\centroids_hog.txt'
centroids_mbhx_handle = 'F:\PR_Term_Project\Computed_features\centroids_mbhx.txt'
centroids_mbhy_handle = 'F:\PR_Term_Project\Computed_features\centroids_mbhy.txt'

encoded_hog = open('F:\PR_Term_Project\Computed_features\encoded_hog.txt', 'w+')
encoded_mbhx = open('F:\PR_Term_Project\Computed_features\encoded_mbhx.txt', 'w+')
encoded_mbhy = open('F:\PR_Term_Project\Computed_features\encoded_mbhy.txt', 'w+')
labels_handle = open('F:\PR_Term_Project\Computed_features\encoded_labels.txt', 'w+')

print('<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<Performing encoding of HOG features>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
perform_encoding(folder_path, centroids_hog_handle, hog_hist, (40, 136), True, labels_array)
print('<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<Performing encoding of MBHx features>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
perform_encoding(folder_path, centroids_mbhx_handle, mbhx_hist, (244, 340))
print('<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<Performing encoding of MBHy features>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
perform_encoding(folder_path, centroids_mbhy_handle, mbhy_hist, (340, -1))

np.savetxt(encoded_hog, hog_hist, delimiter='\t', fmt='%1.7f')
np.savetxt(encoded_mbhx, mbhx_hist, delimiter='\t', fmt='%1.7f')
np.savetxt(encoded_mbhy, mbhy_hist, delimiter='\t', fmt='%1.7f')
np.savetxt(labels_handle, labels_array, delimiter='\t', fmt='%3d')

encoded_hog.close()
encoded_mbhx.close()
encoded_mbhy.close()
labels_handle.close()
print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<Done>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")

<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<Performing encoding of HOG features>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Processed file#  0
Processed file#  1
Processed file#  2
Processed file#  3
Processed file#  4
Processed file#  5
Processed file#  6
Processed file#  7
Processed file#  8
Processed file#  9
Processed file#  10
Processed file#  11
Processed file#  12
Processed file#  13
Processed file#  14
Processed file#  15
Processed file#  16
Processed file#  17
Processed file#  18
Processed file#  19
Processed file#  20
Processed file#  21
Processed file#  22
Processed file#  23
Processed file#  24
Processed file#  25
Processed file#  26
Processed file#  27
Processed file#  28
Processed file#  29
Processed file#  30
Processed file#  31
Processed file#  32
Processed file#  33
Processed file#  34
Processed file#  35
Processed file#  36
Processed file#  37
Processed file#  38
Processed file#  39
Processed file#  40
Processed file#  41
Processed file#  42
Processed file#  43
Processed file# 

Processed file#  390
Processed file#  391
Processed file#  392
Processed file#  393
Processed file#  394
Processed file#  395
Processed file#  396
Processed file#  397
Processed file#  398
Processed file#  399
Processed file#  400
Processed file#  401
Processed file#  402
Processed file#  403
Processed file#  404
Processed file#  405
Processed file#  406
Processed file#  407
Processed file#  408
Processed file#  409
Processed file#  410
Processed file#  411
Processed file#  412
Processed file#  413
Processed file#  414
Processed file#  415
Processed file#  416
Processed file#  417
Processed file#  418
Processed file#  419
Processed file#  420
Processed file#  421
Processed file#  422
Processed file#  423
Processed file#  424
Processed file#  425
Processed file#  426
Processed file#  427
Processed file#  428
Processed file#  429
Processed file#  430
Processed file#  431
Processed file#  432
Processed file#  433
Processed file#  434
Processed file#  435
Processed file#  436
Processed fil

Processed file#  781
Processed file#  782
Processed file#  783
Processed file#  784
Processed file#  785
Processed file#  786
Processed file#  787
Processed file#  788
Processed file#  789
Processed file#  790
Processed file#  791
Processed file#  792
Processed file#  793
Processed file#  794
Processed file#  795
Processed file#  796
Processed file#  797
Processed file#  798
Processed file#  799
Processed file#  800
Processed file#  801
Processed file#  802
Processed file#  803
Processed file#  804
Processed file#  805
Processed file#  806
Processed file#  807
Processed file#  808
Processed file#  809
Processed file#  810
Processed file#  811
Processed file#  812
Processed file#  813
Processed file#  814
Processed file#  815
Processed file#  816
Processed file#  817
Processed file#  818
Processed file#  819
Processed file#  820
Processed file#  821
Processed file#  822
Processed file#  823
Processed file#  824
Processed file#  825
Processed file#  826
Processed file#  827
Processed fil

Processed file#  1164
Processed file#  1165
Processed file#  1166
Processed file#  1167
Processed file#  1168
Processed file#  1169
Processed file#  1170
Processed file#  1171
Processed file#  1172
Processed file#  1173
Processed file#  1174
Processed file#  1175
Processed file#  1176
Processed file#  1177
Processed file#  1178
Processed file#  1179
Processed file#  1180
Processed file#  1181
Processed file#  1182
Processed file#  1183
Processed file#  1184
Processed file#  1185
Processed file#  1186
Processed file#  1187
Processed file#  1188
Processed file#  1189
Processed file#  1190
Processed file#  1191
Processed file#  1192
Processed file#  1193
Processed file#  1194
Processed file#  1195
Processed file#  1196
Processed file#  1197
Processed file#  1198
Processed file#  1199
Processed file#  1200
Processed file#  1201
Processed file#  1202
Processed file#  1203
Processed file#  1204
Processed file#  1205
Processed file#  1206
Processed file#  1207
Processed file#  1208
Processed 

Processed file#  311
Processed file#  312
Processed file#  313
Processed file#  314
Processed file#  315
Processed file#  316
Processed file#  317
Processed file#  318
Processed file#  319
Processed file#  320
Processed file#  321
Processed file#  322
Processed file#  323
Processed file#  324
Processed file#  325
Processed file#  326
Processed file#  327
Processed file#  328
Processed file#  329
Processed file#  330
Processed file#  331
Processed file#  332
Processed file#  333
Processed file#  334
Processed file#  335
Processed file#  336
Processed file#  337
Processed file#  338
Processed file#  339
Processed file#  340
Processed file#  341
Processed file#  342
Processed file#  343
Processed file#  344
Processed file#  345
Processed file#  346
Processed file#  347
Processed file#  348
Processed file#  349
Processed file#  350
Processed file#  351
Processed file#  352
Processed file#  353
Processed file#  354
Processed file#  355
Processed file#  356
Processed file#  357
Processed fil

Processed file#  702
Processed file#  703
Processed file#  704
Processed file#  705
Processed file#  706
Processed file#  707
Processed file#  708
Processed file#  709
Processed file#  710
Processed file#  711
Processed file#  712
Processed file#  713
Processed file#  714
Processed file#  715
Processed file#  716
Processed file#  717
Processed file#  718
Processed file#  719
Processed file#  720
Processed file#  721
Processed file#  722
Processed file#  723
Processed file#  724
Processed file#  725
Processed file#  726
Processed file#  727
Processed file#  728
Processed file#  729
Processed file#  730
Processed file#  731
Processed file#  732
Processed file#  733
Processed file#  734
Processed file#  735
Processed file#  736
Processed file#  737
Processed file#  738
Processed file#  739
Processed file#  740
Processed file#  741
Processed file#  742
Processed file#  743
Processed file#  744
Processed file#  745
Processed file#  746
Processed file#  747
Processed file#  748
Processed fil

Processed file#  1088
Processed file#  1089
Processed file#  1090
Processed file#  1091
Processed file#  1092
Processed file#  1093
Processed file#  1094
Processed file#  1095
Processed file#  1096
Processed file#  1097
Processed file#  1098
Processed file#  1099
Processed file#  1100
Processed file#  1101
Processed file#  1102
Processed file#  1103
Processed file#  1104
Processed file#  1105
Processed file#  1106
Processed file#  1107
Processed file#  1108
Processed file#  1109
Processed file#  1110
Processed file#  1111
Processed file#  1112
Processed file#  1113
Processed file#  1114
Processed file#  1115
Processed file#  1116
Processed file#  1117
Processed file#  1118
Processed file#  1119
Processed file#  1120
Processed file#  1121
Processed file#  1122
Processed file#  1123
Processed file#  1124
Processed file#  1125
Processed file#  1126
Processed file#  1127
Processed file#  1128
Processed file#  1129
Processed file#  1130
Processed file#  1131
Processed file#  1132
Processed 

Processed file#  231
Processed file#  232
Processed file#  233
Processed file#  234
Processed file#  235
Processed file#  236
Processed file#  237
Processed file#  238
Processed file#  239
Processed file#  240
Processed file#  241
Processed file#  242
Processed file#  243
Processed file#  244
Processed file#  245
Processed file#  246
Processed file#  247
Processed file#  248
Processed file#  249
Processed file#  250
Processed file#  251
Processed file#  252
Processed file#  253
Processed file#  254
Processed file#  255
Processed file#  256
Processed file#  257
Processed file#  258
Processed file#  259
Processed file#  260
Processed file#  261
Processed file#  262
Processed file#  263
Processed file#  264
Processed file#  265
Processed file#  266
Processed file#  267
Processed file#  268
Processed file#  269
Processed file#  270
Processed file#  271
Processed file#  272
Processed file#  273
Processed file#  274
Processed file#  275
Processed file#  276
Processed file#  277
Processed fil

Processed file#  622
Processed file#  623
Processed file#  624
Processed file#  625
Processed file#  626
Processed file#  627
Processed file#  628
Processed file#  629
Processed file#  630
Processed file#  631
Processed file#  632
Processed file#  633
Processed file#  634
Processed file#  635
Processed file#  636
Processed file#  637
Processed file#  638
Processed file#  639
Processed file#  640
Processed file#  641
Processed file#  642
Processed file#  643
Processed file#  644
Processed file#  645
Processed file#  646
Processed file#  647
Processed file#  648
Processed file#  649
Processed file#  650
Processed file#  651
Processed file#  652
Processed file#  653
Processed file#  654
Processed file#  655
Processed file#  656
Processed file#  657
Processed file#  658
Processed file#  659
Processed file#  660
Processed file#  661
Processed file#  662
Processed file#  663
Processed file#  664
Processed file#  665
Processed file#  666
Processed file#  667
Processed file#  668
Processed fil

Processed file#  1012
Processed file#  1013
Processed file#  1014
Processed file#  1015
Processed file#  1016
Processed file#  1017
Processed file#  1018
Processed file#  1019
Processed file#  1020
Processed file#  1021
Processed file#  1022
Processed file#  1023
Processed file#  1024
Processed file#  1025
Processed file#  1026
Processed file#  1027
Processed file#  1028
Processed file#  1029
Processed file#  1030
Processed file#  1031
Processed file#  1032
Processed file#  1033
Processed file#  1034
Processed file#  1035
Processed file#  1036
Processed file#  1037
Processed file#  1038
Processed file#  1039
Processed file#  1040
Processed file#  1041
Processed file#  1042
Processed file#  1043
Processed file#  1044
Processed file#  1045
Processed file#  1046
Processed file#  1047
Processed file#  1048
Processed file#  1049
Processed file#  1050
Processed file#  1051
Processed file#  1052
Processed file#  1053
Processed file#  1054
Processed file#  1055
Processed file#  1056
Processed 